In [1]:
import datasets
import json
import os
import numpy as np
import collections
import re
import string
import math

In [2]:
# Change these variables to perform evalution for you task
test_path = os.path.abspath(os.getcwd()).split('macaw_evaluation_scripts')[0] + 'processed_data/macaw/completion_4/processed_test.json'
generated_path = os.path.abspath(os.getcwd()).split('macaw_evaluation_scripts')[0] + 'generated_data_macaw/experiment_4/generated_answers.json'
distractor_path = os.path.abspath(os.getcwd()).split('macaw_evaluation_scripts')[0] + 'generated_data_macaw/experiment_4/generated_distractors.json'

In [3]:
# Get all the target prompts and completions
test_data = []

for line in open(test_path):
    test_data.append((json.loads(line)))
    
generated_data = []

# Get the generated completions
with open(generated_path) as f:
    generated = json.load(f)

for key in generated:
    generated_data.append(generated[key])
    
generated_distractors = []

# Get the generated completions
with open(distractor_path) as f:
    generated = json.load(f)

for key in generated:
    generated_distractors.append(generated[key])

In [4]:
count = 0
similar_count = 0

predictions = []
gold_references = []

for i in range(len(test_data)):
    count += 1
    
    completion = test_data[i]['prompt'].split("$answer$ = ")[1].split(" $context$")[0]
    clean_completion = completion.replace(" ","").replace("~","").replace('.',"").lower()
    
    generated_completion = generated_data[i][0].split('= ')[-1]
    clean_generated_completion = generated_completion.replace(" ","").replace('.',"").lower()
    
    if clean_completion == clean_generated_completion:
        similar_count += 1
        
    predictions.append(generated_completion)
    gold_references.append(completion)
    
print('Accuracy:', round(similar_count/count * 100, 2), '%')

Accuracy: 76.0 %


In [5]:
# Create result files

with open(os.path.abspath(os.getcwd()).split('macaw_evaluation_scripts')[0] + 'generated_data_macaw/experiment_4/' + 'predictions.txt', 'w') as f:
    for i in range(len(predictions)):
        f.write(predictions[i] + '\n')
        
with open(os.path.abspath(os.getcwd()).split('macaw_evaluation_scripts')[0] + 'generated_data_macaw/experiment_4/' + 'ground_truth.txt', 'w') as f:
    for i in range(len(gold_references)):
        f.write(gold_references[i] + '\n')
        
with open(os.path.abspath(os.getcwd()).split('macaw_evaluation_scripts')[0] + 'generated_data_macaw/experiment_4/' + 'automatic_evaluation.txt', 'w') as f:
    f.write('Accuracy: ' + str(round(similar_count/count * 100, 2)) + '%')

In [6]:
count = 0
with open(os.path.abspath(os.getcwd()).split('macaw_evaluation_scripts')[0] + 'human_evaluation/experiment4/' + 'exp4_macaw.txt', 'w') as f:
    for i in range(0,len(test_data),math.floor(len(test_data)/100)):
        temp = test_data[i]['completion'].split('\n')
        mcoptions = generated_distractors[i][0].split('= ')[-1].split('(')
        f.write('Excel row: ' + str(count+2) + ' Test instance: ' + str(i+1) + '\n\n')
        f.write(test_data[i]['prompt'].split("$context$ = ")[1] + '\n\n')
        f.write("Question: " + test_data[i]['prompt'].split("$question$ = ")[1].split("$answer")[0] + '\n\n')
        f.write("Answer: " + test_data[i]['prompt'].split("$answer$ = ")[1].split("$context")[0] + '\n\n')
        f.write('Generated: ' + '\n')
        f.write('MC option ' + mcoptions[1] + '\n')
        f.write('MC option ' + mcoptions[2] + '\n')
        f.write('MC option ' + mcoptions[3] + '\n')
        f.write('MC option ' + mcoptions[4] + '\n\n')
        f.write('----------------------------------------------------------------------------------------' + '\n\n')
        count+=1
        
        if count == 100:
            break